In [ ]:
from kiteconnect import KiteConnect
import xlwings as xw
import time
import datetime

kite = KiteConnect(api_key="v6khvkcvmrjba7fb")

In [ ]:
request_token = "u4NB61Oizrr1hIxJrd3ga8aY4cE6Y3oy"
data = kite.generate_session(request_token, api_secret="81dy7evzpi657r9bpz6civuydv5p35xg")

In [ ]:
def save_access_token(token, filename="access_token.txt"):
    with open(filename, "w") as f:
        f.write(token)

access_token = data["access_token"]
save_access_token(access_token)

In [ ]:
def load_access_token(filename="access_token.txt"):
    try:
        with open(filename, "r") as f:
            token = f.read().strip()
        return token
    except FileNotFoundError:
        return None

# Usage:
access_token = load_access_token()
if not access_token:
    raise Exception("Access token file not found. Please login and save token first.")

kite.set_access_token(access_token)

In [ ]:
from kiteconnect import KiteConnect
from kiteconnect.exceptions import KiteException
import xlwings as xw
import time
import datetime

EXCEL_FILE = "options_live.xlsx"
EXPECTED_COLS = 24  # Adjust if you add/remove columns

kite = KiteConnect(api_key="v6khvkcvmrjba7fb")

def load_access_token(filename="access_token.txt"):
    try:
        with open(filename, "r") as f:
            token = f.read().strip()
        return token
    except FileNotFoundError:
        return None

access_token = load_access_token()
if not access_token:
    raise Exception("Access token file not found. Please login and save token first.")
kite.set_access_token(access_token)

def get_default_lot_size(symbol):
    symbol = symbol.upper()
    if "NIFTY" in symbol and "BANK" not in symbol and "MID" not in symbol and "FIN" not in symbol and "NEXT" not in symbol:
        return 75
    elif "BANKNIFTY" in symbol:
        return 30
    elif "FINNIFTY" in symbol:
        return 65
    elif "MIDCPNIFTY" in symbol or "NIFTY MIDCAP SELECT" in symbol:
        return 140
    elif "NEXT" in symbol:
        return 25
    elif "SENSEX" in symbol:
        return 20
    else:
        return ""

def fetch_option_data(symbol):
    try:
        quotes = kite.quote([symbol])
        data = quotes.get(symbol, {})
        return [
            data.get("ohlc", {}).get("open", ""),
            data.get("ohlc", {}).get("high", ""),
            data.get("ohlc", {}).get("low", ""),
            data.get("last_price", ""),
            data.get("volume", ""),
            data.get("average_price", ""),
            data.get("depth", {}).get("buy", [{}])[0].get("price", ""),
            data.get("depth", {}).get("sell", [{}])[0].get("price", ""),
            data.get("ohlc", {}).get("close", "")
        ]
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return [""] * 9

def fetch_order_status(order_id):
    try:
        order_history = kite.order_history(order_id=order_id)
        if order_history:
            last_update = order_history[-1]
            status = last_update['status']
            filled_qty = last_update['filled_quantity']
            avg_price = last_update['average_price']
            return status, filled_qty, avg_price
        else:
            return "UNKNOWN", 0, 0
    except Exception as e:
        print(f"Error fetching order status: {e}")
        return "ERROR", 0, 0

def calculate_pnl(buy_price, ltp, qty):
    try:
        return round((ltp - buy_price) * qty, 2)
    except:
        return ""

def pad_row(row):
    if row is None:
        return [None] * EXPECTED_COLS
    return list(row) + [None] * (EXPECTED_COLS - len(row))

def fetch_holdings():
    try:
        holdings = kite.holdings()
        return holdings
    except Exception as e:
        print(f"Error fetching holdings: {e}")
        return []

def update_portfolio_sheet(s2, positions):
    # --- Write positions (batch) ---
    pos_rows = [
        [
            pos.get('instrument_token'),
            pos.get('tradingsymbol'),
            pos.get('exchange'),
            pos.get('quantity'),
            pos.get('average_price'),
            pos.get('last_price'),
            pos.get('unrealised'),
            pos.get('realised'),
            pos.get('day_change')
        ]
        for pos in positions
    ]
    n_positions = len(pos_rows)
    max_pos_rows = 200  # Adjust as needed

    if n_positions > 0:
        s2.range("A2").options(expand='table').value = pos_rows
        if n_positions < max_pos_rows:
            s2.range(f"A{n_positions+2}:I{max_pos_rows+1}").value = [[""]*9]*(max_pos_rows-n_positions)
    else:
        s2.range("A2:I201").value = [[""]*9]*200

    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    s2.range("K1").value = f"Last Updated: {now}"

def update_holdings_sheet(s3, holdings, kite):
    holding_headers = [
        "Trading Symbol", "Exchange", "ISIN", "Quantity", "T1 Qty", "Average Price",
        "Last Price (Live)", "Close Price", "PnL", "Day Change", "Day Change %"
    ]
    s3.range("A1").value = holding_headers

    holding_rows = []
    symbols = []
    for h in holdings:
        if h.get("exchange") and h.get("tradingsymbol"):
            symbols.append(f"{h['exchange']}:{h['tradingsymbol']}")
    # Batch fetch live prices
    live_prices = {}
    if symbols:
        try:
            quotes = kite.quote(symbols)
            for sym in symbols:
                live_prices[sym] = quotes.get(sym, {}).get("last_price", "")
        except Exception as e:
            print(f"Error fetching live prices for holdings: {e}")

    for h in holdings:
        sym = f"{h.get('exchange')}:{h.get('tradingsymbol')}"
        live_last_price = live_prices.get(sym, h.get("last_price"))
        holding_rows.append([
            h.get("tradingsymbol"),
            h.get("exchange"),
            h.get("isin"),
            h.get("quantity"),
            h.get("t1_quantity"),
            h.get("average_price"),
            live_last_price,
            h.get("close_price"),
            h.get("pnl"),
            h.get("day_change"),
            h.get("day_change_percentage"),
        ])
    n_holdings = len(holding_rows)
    max_hold_rows = 200  # Adjust as needed

    if n_holdings > 0:
        s3.range("A2").options(expand='table').value = holding_rows
        if n_holdings < max_hold_rows:
            s3.range(f"A{n_holdings+2}:K{max_hold_rows+1}").value = [[""]*11]*(max_hold_rows-n_holdings)
    else:
        s3.range("A2:K201").value = [[""]*11]*200

    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    s3.range("L1").value = f"Last Updated: {now}"

In [ ]:
def monitor_excel():
    wb = xw.Book(EXCEL_FILE)
    sheet = wb.sheets['INPUT']
    print("Monitoring Excel for symbols and orders... (Ctrl+C to stop)")
    order_id_map = {}
    s2 = wb.sheets['Portfolio']

    headers = [
        "Instrument Token", "Trading Symbol", "Exchange", "Quantity",
        "Average Price", "Last Price", "Unrealized PnL", "Realized PnL", "Day's PnL"
    ]
    s2.range("A1").value = headers
    s2.range("A1:I1").api.Font.Bold = True

    while True:
        try:
            # Batch read all data
            data = sheet.range("A2:X201").value  # Adjust X and 201 as per your sheet
        except Exception as e:
            print(f"Excel read error: {e}")
            time.sleep(5)
            continue

        if not data:
            time.sleep(5)
            continue

        data = [pad_row(row) for row in data]

        # Prepare batch updates
        price_updates = []        # For columns B–J (open, high, ...)
        last_updated_updates = [] # For column P (Last Updated)
        entry_updates = []        # For column O (Entry)
        pnl_updates = []          # For column Q (PnL)

        now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        for i, row in enumerate(data):
            idx = i + 2  # Excel row number (since data starts at row 2)
            symbol = row[0]
            qty = row[10]
            direction = row[11]
            entry_signal = row[12]
            exit_signal = row[13]
            entry_status = row[14]
            variety = row[17]
            order_type = row[18]
            product = row[19]
            validity = row[20]
            price = row[21]
            trigger_price = row[22]
            ttl_minutes = row[23]

            if symbol and isinstance(symbol, str):
                symbol = symbol.upper()
                option_data = fetch_option_data(symbol)
                price_updates.append(option_data)
                last_updated_updates.append([now])
            else:
                price_updates.append([""]*9)
                last_updated_updates.append([""])

            # --- Order placement and status ---
            entry_cell = ""
            pnl_cell = ""
            try:
                if symbol and not qty:
                    default_qty = get_default_lot_size(symbol)
                    if default_qty:
                        sheet.range(f"K{idx}").value = default_qty
                        qty = default_qty

                # Place order if signal is yes and entry_status is empty
                if (symbol and qty and direction and entry_signal and variety and order_type and product and validity and
                    str(entry_signal).strip().lower() == "yes" and (not entry_status or entry_status == "")):
                    try:
                        exchange, tradingsymbol = symbol.split(":")
                        order_params = dict(
                            variety=variety.strip().lower(),
                            exchange=exchange,
                            tradingsymbol=tradingsymbol,
                            transaction_type=direction.strip().upper(),
                            quantity=int(qty),
                            order_type=order_type.strip().upper(),
                            product=product.strip().upper(),
                            validity=validity.strip().upper()
                        )
                        if order_type.strip().upper() in ["LIMIT", "SL", "SL-M"] and price:
                            order_params["price"] = float(price)
                        if order_type.strip().upper() in ["SL", "SL-M"] and trigger_price:
                            order_params["trigger_price"] = float(trigger_price)
                        if validity.strip().upper() == "TTL" and ttl_minutes:
                            order_params["validity_ttl"] = int(ttl_minutes)

                        order_id = kite.place_order(**order_params)
                        order_id_map[idx] = order_id
                        time.sleep(5)
                        status, filled_qty, avg_price = fetch_order_status(order_id)
                        if status in ["TRIGGER PENDING", "OPEN"]:
                            entry_cell = f"TRIGGER PENDING ({filled_qty})"
                        elif status == "COMPLETE":
                            entry_cell = f"ORDERED ({filled_qty})"
                        elif status == "PARTIAL":
                            entry_cell = f"PARTIAL ({filled_qty})"
                        elif status in ["REJECTED", "CANCELLED"]:
                            entry_cell = f"ERROR: {status}"
                        else:
                            entry_cell = f"PENDING ({filled_qty})"
                        ltp = row[4]
                        if filled_qty and avg_price and ltp:
                            pnl_cell = calculate_pnl(avg_price, float(ltp), int(filled_qty))
                    except KiteException as e:
                        print(f"Order placement failed: {str(e)}")
                        entry_cell = f"ERROR: {str(e)}"
                    except Exception as e:
                        print(f"Order placement failed: {e}")
                        entry_cell = f"ERROR: {str(e)}"
                # Poll and update status for open orders
                elif entry_status and isinstance(entry_status, str) and (
                    "TRIGGER PENDING" in entry_status or "PENDING" in entry_status or "OPEN" in entry_status):
                    order_id = order_id_map.get(idx)
                    if order_id:
                        status, filled_qty, avg_price = fetch_order_status(order_id)
                        if status in ["TRIGGER PENDING", "OPEN"]:
                            entry_cell = f"TRIGGER PENDING ({filled_qty})"
                        elif status == "COMPLETE":
                            entry_cell = f"ORDERED ({filled_qty})"
                        elif status == "PARTIAL":
                            entry_cell = f"PARTIAL ({filled_qty})"
                        elif status in ["REJECTED", "CANCELLED"]:
                            entry_cell = f"ERROR: {status}"
                        else:
                            entry_cell = f"PENDING ({filled_qty})"
                        ltp = row[4]
                        if filled_qty and avg_price and ltp:
                            pnl_cell = calculate_pnl(avg_price, float(ltp), int(filled_qty))
                # Place exit order if needed
                elif (symbol and qty and direction and exit_signal and
                    str(exit_signal).strip().lower() == "yes" and entry_status and "ORDERED" in str(entry_status)):
                    exit_direction = "SELL" if direction.strip().upper() == "BUY" else "BUY"
                    try:
                        exchange, tradingsymbol = symbol.split(":")
                        order_id = kite.place_order(
                            variety=variety.strip().lower(),
                            exchange=exchange,
                            tradingsymbol=tradingsymbol,
                            transaction_type=exit_direction,
                            quantity=int(qty),
                            order_type="MARKET",
                            product=product.strip().upper(),
                            validity=validity.strip().upper()
                        )
                        time.sleep(5)
                        status, filled_qty, avg_price = fetch_order_status(order_id)
                        if status == "COMPLETE":
                            entry_cell = f"EXITED ({filled_qty})"
                        elif status == "PARTIAL":
                            entry_cell = f"PARTIAL EXIT ({filled_qty})"
                        elif status in ["REJECTED", "CANCELLED"]:
                            entry_cell = f"ERROR: {status}"
                        else:
                            entry_cell = f"PENDING EXIT ({filled_qty})"
                        ltp = row[4]
                        if filled_qty and avg_price and ltp:
                            pnl_cell = calculate_pnl(avg_price, float(ltp), int(filled_qty))
                    except KiteException as e:
                        print(f"Exit order failed: {str(e)}")
                        entry_cell = f"ERROR: {str(e)}"
                    except Exception as e:
                        print(f"Exit order failed: {e}")
                        entry_cell = f"ERROR: {str(e)}"
            except Exception as e:
                print(f"Order failed for {symbol}: {e}")
                entry_cell = f"ERROR: {e}"

            entry_updates.append([entry_cell])
            pnl_updates.append([pnl_cell])

        # --- Batch write all updates back to Excel ---
        sheet.range("B2:J201").value = price_updates
        sheet.range("P2:P201").value = last_updated_updates
        sheet.range("O2:O201").value = entry_updates
        sheet.range("Q2:Q201").value = pnl_updates

        try:
            positions = kite.positions().get('net', [])
            holdings = fetch_holdings()

            # Portfolio sheet (positions)
            update_portfolio_sheet(wb.sheets['Portfolio'], positions)

            # Holdings sheet (live, no clear, always fresh)
            if 'Holdings' not in [sh.name for sh in wb.sheets]:
                wb.sheets.add('Holdings')
            s3 = wb.sheets['Holdings']
            update_holdings_sheet(s3, holdings, kite)

        except Exception as e:
            print(f"Error updating portfolio/holdings: {e}")


        time.sleep(5)

if __name__ == "__main__":
    monitor_excel()

In [ ]:
import xlwings as xw
import time
import datetime

EXCEL_FILE = "options_live.xlsx"
PORTFOLIO_SHEET = "Portfolio"
MAX_ROWS = 200 

def fetch_positions_and_prices():
    positions = kite.positions().get('net', [])
    # Build list of symbols for quote API
    symbols = []
    for pos in positions:
        exch = pos.get('exchange')
        sym = pos.get('tradingsymbol')
        if exch and sym:
            symbols.append(f"{exch}:{sym}")
    # Fetch all live prices in batch
    live_prices = {}
    if symbols:
        try:
            quotes = kite.quote(symbols)
            for sym in symbols:
                live_prices[sym] = quotes.get(sym, {}).get("last_price", "")
        except Exception as e:
            print(f"Error fetching live prices: {e}")
    return positions, live_prices

def update_portfolio_sheet(sht, positions, live_prices):
    # Prepare data rows
    rows = []
    for pos in positions:
        exch = pos.get('exchange')
        sym = pos.get('tradingsymbol')
        symbol_key = f"{exch}:{sym}"
        last_price = live_prices.get(symbol_key, pos.get('last_price', ""))
        rows.append([
            pos.get('instrument_token'),
            sym,
            exch,
            pos.get('quantity'),
            pos.get('average_price'),
            last_price,
            pos.get('unrealised'),
            pos.get('realised'),
            pos.get('day_change')
        ])
    n = len(rows)
    # Write data rows
    if n > 0:
        sht.range("A2").options(expand='table').value = rows
        # Blank out leftover rows if any
        if n < MAX_ROWS:
            sht.range(f"A{n+2}:I{MAX_ROWS+1}").value = [[""]*9]*(MAX_ROWS-n)
    else:
        sht.range("A2:I201").value = [[""]*9]*MAX_ROWS
    # Update Last Updated cell
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    sht.range("K1").value = f"Last Updated: {now}"

def main():
    wb = xw.Book(EXCEL_FILE)
    if PORTFOLIO_SHEET not in [s.name for s in wb.sheets]:
        wb.sheets.add(PORTFOLIO_SHEET)
    sht = wb.sheets[PORTFOLIO_SHEET]
    # Write headers if not present
    headers = [
        "Instrument Token", "Trading Symbol", "Exchange", "Quantity",
        "Average Price", "Last Price", "Unrealized PnL", "Realized PnL", "Day's PnL"
    ]
    sht.range("A1").value = headers
    sht.range("A1:I1").api.Font.Bold = True

    while True:
        try:
            positions, live_prices = fetch_positions_and_prices()
            update_portfolio_sheet(sht, positions, live_prices)
        except Exception as e:
            print(f"Error updating portfolio: {e}")
        time.sleep(5)

if __name__ == "__main__":
    main()

In [ ]:
from kiteconnect import KiteConnect
from kiteconnect.exceptions import KiteException
import xlwings as xw
import time
import datetime

EXCEL_FILE = "options_live.xlsx"
EXPECTED_COLS = 25
MAX_POS_ROWS = 200
MAX_HOLD_ROWS = 200
MAX_ORDERS_ROWS = 2000
API_KEY = "v6khvkcvmrjba7fb"
ACCESS_TOKEN_FILE = "access_token.txt"

def load_access_token(filename=ACCESS_TOKEN_FILE):
    with open(filename, "r") as f:
        return f.read().strip()

def pad_row(row):
    if row is None:
        return [None] * EXPECTED_COLS
    return list(row) + [None] * (EXPECTED_COLS - len(row))

def get_default_lot_size(symbol):
    symbol = symbol.upper()
    if "NIFTY" in symbol and "BANK" not in symbol and "MID" not in symbol and "FIN" not in symbol and "NEXT" not in symbol:
        return 75
    elif "BANKNIFTY" in symbol:
        return 30
    elif "FINNIFTY" in symbol:
        return 65
    elif "MIDCPNIFTY" in symbol or "NIFTY MIDCAP SELECT" in symbol:
        return 140
    elif "NEXT" in symbol:
        return 25
    elif "SENSEX" in symbol:
        return 20
    else:
        return 1

def fetch_order_status(kite, order_id):
    try:
        order_history = kite.order_history(order_id=order_id)
        if order_history:
            last_update = order_history[-1]
            status = last_update['status']
            filled_qty = last_update['filled_quantity']
            avg_price = last_update['average_price']
            return status, filled_qty, avg_price
        else:
            return "UNKNOWN", 0, 0
    except Exception as e:
        print(f"Error fetching order status: {e}")
        return "ERROR", 0, 0

def calculate_pnl(buy_price, ltp, qty):
    try:
        return round((ltp - buy_price) * qty, 2)
    except:
        return ""

def fetch_holdings(kite):
    try:
        return kite.holdings()
    except Exception as e:
        print(f"Error fetching holdings: {e}")
        return []

def update_orders_sheet(orders_sheet, kite):
    orders = kite.orders()
    headers = [
        "Order ID", "Variety", "Status", "Trading Symbol", "Exchange", "Order Type",
        "Product", "Transaction Type", "Quantity", "Price", "Trigger Price",
        "Average Price", "Pending Quantity", "Filled Quantity", "Order Timestamp",
        "Modify", "Cancel", "New Price", "New Trigger"
    ]
    orders_sheet.range("A1").value = headers
    n = len(orders)
    for i, o in enumerate(orders):
        row = [
            o.get("order_id"),
            o.get("variety"),
            o.get("status"),
            o.get("tradingsymbol"),
            o.get("exchange"),
            o.get("order_type"),
            o.get("product"),
            o.get("transaction_type"),
            o.get("quantity"),
            o.get("price"),
            o.get("trigger_price"),
            o.get("average_price"),
            o.get("pending_quantity"),
            o.get("filled_quantity"),
            o.get("order_timestamp"),
        ]
        orders_sheet.range(f"A{i+2}:O{i+2}").value = row
    if n < MAX_ORDERS_ROWS:
        orders_sheet.range(f"A{n+2}:O{MAX_ORDERS_ROWS+1}").value = [[""]*15]*(MAX_ORDERS_ROWS-n)

def process_order_modifications(orders_sheet, kite, force_refresh=False):
    def is_one(val):
        if val is None:
            return False
        if isinstance(val, (int, float)) and val == 1:
            return True
        if isinstance(val, str) and val.strip().lower() in ["yes", "y", "true", "1"]:
            return True
        return False
    try:
        if orders_sheet.range("R1").value is None:
            orders_sheet.range("R1").value = "New Price"
            orders_sheet.range("S1").value = "New Trigger"
            orders_sheet.range("R1:S1").api.Font.Bold = True
        if force_refresh:
            try:
                app = xw.apps.active
                if app:
                    app.calculate()
            except Exception as e:
                print(f"Could not force Excel calculation: {e}")
        data = orders_sheet.range("A2:Q201").value
        for i, row in enumerate(data):
            if not row or not row[0]:
                continue
            order_id = row[0]
            variety = row[1]
            status = row[2]
            order_type = row[5]
            quantity = row[8]
            modify_col = row[15]
            cancel_col = row[16]
            new_price = orders_sheet.range(f"R{i+2}").value
            new_trigger_price = orders_sheet.range(f"S{i+2}").value
            order_id_str = str(int(order_id)) if isinstance(order_id, float) else str(order_id)
            if is_one(modify_col):
                if status not in ["OPEN", "TRIGGER PENDING", "PENDING", "AMO REQ RECEIVED", "AMO REQ PROCESSING"]:
                    orders_sheet.range(f"P{i+2}").value = "Not modifiable"
                    continue
                try:
                    variety_str = str(variety).strip().lower() if variety else 'regular'
                    params = {
                        "variety": variety_str,
                        "order_id": order_id_str,
                        "quantity": int(quantity) if quantity else None,
                        "order_type": str(order_type).upper() if order_type else None,
                        "validity": "DAY"
                    }
                    if order_type and str(order_type).upper() in ["SL", "SL-M"]:
                        if new_price is not None: params["price"] = float(new_price)
                        if new_trigger_price is not None: params["trigger_price"] = float(new_trigger_price)
                    else:
                        if new_price is not None: params["price"] = float(new_price)
                    params = {k: v for k, v in params.items() if v is not None}
                    if "price" in params or "trigger_price" in params:
                        try:
                            kite.modify_order(**params)
                        except Exception as api_e:
                            orders_sheet.range(f"P{i+2}").value = f"ERROR: {api_e}"
                            continue
                        time.sleep(5)
                        try:
                            order_hist = kite.order_history(order_id_str)
                            last_update = order_hist[-1]
                            status_msg = last_update.get("status_message")
                            if status_msg:
                                orders_sheet.range(f"P{i+2}").value = status_msg
                            else:
                                orders_sheet.range(f"P{i+2}").value = f"Modified: P={last_update.get('price')}, TP={last_update.get('trigger_price')}"
                        except Exception as hist_e:
                            pass
                        orders_sheet.range(f"R{i+2}").value = None
                        orders_sheet.range(f"S{i+2}").value = None
                    else:
                        orders_sheet.range(f"P{i+2}").value = "ERROR: No new price/trigger provided"
                except Exception as e:
                    orders_sheet.range(f"P{i+2}").value = f"ERROR: {str(e)}"
            if is_one(cancel_col):
                if status not in ["OPEN", "TRIGGER PENDING", "PENDING", "AMO REQ RECEIVED", "AMO REQ PROCESSING"]:
                    orders_sheet.range(f"Q{i+2}").value = "Not cancellable"
                    continue
                try:
                    variety_str = str(variety).strip().lower() if variety else 'regular'
                    try:
                        kite.cancel_order(variety=variety_str, order_id=order_id_str)
                    except Exception as api_e:
                        orders_sheet.range(f"Q{i+2}").value = f"ERROR: {api_e}"
                        continue
                    time.sleep(5)
                    try:
                        order_hist = kite.order_history(order_id_str)
                        last_update = order_hist[-1]
                        status_msg = last_update.get("status_message")
                        if status_msg:
                            orders_sheet.range(f"Q{i+2}").value = status_msg
                        else:
                            orders_sheet.range(f"Q{i+2}").value = "Cancelled (check orderbook)"
                        orders_sheet.range(f"P{i+2}").value = ""
                    except Exception as hist_e:
                        pass
                except Exception as e:
                    orders_sheet.range(f"Q{i+2}").value = f"ERROR: {str(e)}"
    except Exception as e:
        print(f"Error in process_order_modifications: {str(e)}")

def update_input_sheet(sheet, kite, order_id_map, quotes, positions=None):
    if positions is None:
        positions = kite.positions().get('net', [])
    pos_qty_map = {}
    for pos in positions:
        exch = pos.get('exchange')
        sym = pos.get('tradingsymbol')
        qty = pos.get('quantity', 0)
        if exch and sym:
            pos_qty_map[f"{exch}:{sym}"] = qty
    data = sheet.range("A2:Y201").value
    if not data:
        data = []
    data = [row if isinstance(row, list) else [row] for row in data]
    while len(data) < 200:
        data.append([None]*25)
    price_updates = []
    portf_qty_updates = []
    last_updated_updates = []
    entry_updates = []
    pnl_updates = []
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    for i, row in enumerate(data):
        row = (row + [None]*25)[:25]
        symbol = row[0]
        qty = row[11]
        direction = row[12]
        entry_signal = row[13]
        exit_signal = row[14]
        entry_status = row[15]
        variety = row[18]
        order_type = row[19]
        product = row[20]
        validity = row[21]
        price = row[22]
        trigger_price = row[23]
        ttl_minutes = row[24]
        # Portfolio Qty
        portfolio_qty = ""
        if symbol and isinstance(symbol, str) and ":" in symbol:
            symbol_key = symbol.upper()
            portfolio_qty = pos_qty_map.get(symbol_key, "")
        portf_qty_updates.append([portfolio_qty])
        # Price columns (C-K, 9 cols)
        if symbol and isinstance(symbol, str):
            symbol = symbol.upper()
            q = quotes.get(symbol, {})
            option_data = [
                q.get("ohlc", {}).get("open", ""),
                q.get("ohlc", {}).get("high", ""),
                q.get("ohlc", {}).get("low", ""),
                q.get("last_price", ""),
                q.get("volume", ""),
                q.get("average_price", ""),
                q.get("depth", {}).get("buy", [{}])[0].get("price", ""),
                q.get("depth", {}).get("sell", [{}])[0].get("price", ""),
                q.get("ohlc", {}).get("close", "")
            ]
            price_updates.append(option_data)
            last_updated_updates.append([now])
        else:
            price_updates.append([""]*9)
            last_updated_updates.append([""])
        entry_cell = ""
        pnl_cell = ""
        try:
            if symbol and not qty:
                default_qty = get_default_lot_size(symbol)
                if default_qty:
                    sheet.range(f"L{i+2}").value = default_qty
                    qty = default_qty
            if (symbol and qty and direction and entry_signal and variety and order_type and product and validity and
                str(entry_signal).strip().lower() == "yes" and (not entry_status or entry_status == "")):
                try:
                    exchange, tradingsymbol = symbol.split(":")
                    order_params = dict(
                        variety=variety.strip().lower(),
                        exchange=exchange,
                        tradingsymbol=tradingsymbol,
                        transaction_type=direction.strip().upper(),
                        quantity=int(qty),
                        order_type=order_type.strip().upper(),
                        product=product.strip().upper(),
                        validity=validity.strip().upper()
                    )
                    if order_type.strip().upper() in ["LIMIT", "SL", "SL-M"] and price:
                        order_params["price"] = float(price)
                    if order_type.strip().upper() in ["SL", "SL-M"] and trigger_price:
                        order_params["trigger_price"] = float(trigger_price)
                    if validity.strip().upper() == "TTL" and ttl_minutes:
                        order_params["validity_ttl"] = int(ttl_minutes)
                    order_id = kite.place_order(**order_params)
                    order_id_map[i+2] = order_id
                    time.sleep(5)
                    status, filled_qty, avg_price = fetch_order_status(order_id)
                    if status in ["TRIGGER PENDING", "OPEN"]:
                        entry_cell = f"TRIGGER PENDING ({filled_qty})"
                    elif status == "COMPLETE":
                        entry_cell = f"ORDERED ({filled_qty})"
                    elif status == "PARTIAL":
                        entry_cell = f"PARTIAL ({filled_qty})"
                    elif status in ["REJECTED", "CANCELLED"]:
                        entry_cell = f"ERROR: {status}"
                    else:
                        entry_cell = f"PENDING ({filled_qty})"
                    ltp = row[5]
                    if filled_qty and avg_price and ltp:
                        pnl_cell = calculate_pnl(avg_price, float(ltp), int(filled_qty))
                except KiteException as e:
                    entry_cell = f"ERROR: {str(e)}"
                except Exception as e:
                    entry_cell = f"ERROR: {str(e)}"
            elif entry_status and isinstance(entry_status, str) and (
                "TRIGGER PENDING" in entry_status or "PENDING" in entry_status or "OPEN" in entry_status):
                order_id = order_id_map.get(i+2)
                if order_id:
                    status, filled_qty, avg_price = fetch_order_status(order_id)
                    if status in ["TRIGGER PENDING", "OPEN"]:
                        entry_cell = f"TRIGGER PENDING ({filled_qty})"
                    elif status == "COMPLETE":
                        entry_cell = f"ORDERED ({filled_qty})"
                    elif status == "PARTIAL":
                        entry_cell = f"PARTIAL ({filled_qty})"
                    elif status in ["REJECTED", "CANCELLED"]:
                        entry_cell = f"ERROR: {status}"
                    else:
                        entry_cell = f"PENDING ({filled_qty})"
                    ltp = row[5]
                    if filled_qty and avg_price and ltp:
                        pnl_cell = calculate_pnl(avg_price, float(ltp), int(filled_qty))
            elif (symbol and qty and direction and exit_signal and
                str(exit_signal).strip().lower() == "yes" and entry_status and "ORDERED" in str(entry_status)):
                exit_direction = "SELL" if direction.strip().upper() == "BUY" else "BUY"
                try:
                    exchange, tradingsymbol = symbol.split(":")
                    order_id = kite.place_order(
                        variety=variety.strip().lower(),
                        exchange=exchange,
                        tradingsymbol=tradingsymbol,
                        transaction_type=exit_direction,
                        quantity=int(qty),
                        order_type="MARKET",
                        product=product.strip().upper(),
                        validity=validity.strip().upper()
                    )
                    time.sleep(5)
                    status, filled_qty, avg_price = fetch_order_status(order_id)
                    if status == "COMPLETE":
                        entry_cell = f"EXITED ({filled_qty})"
                    elif status == "PARTIAL":
                        entry_cell = f"PARTIAL EXIT ({filled_qty})"
                    elif status in ["REJECTED", "CANCELLED"]:
                        entry_cell = f"ERROR: {status}"
                    else:
                        entry_cell = f"PENDING EXIT ({filled_qty})"
                    ltp = row[5]
                    if filled_qty and avg_price and ltp:
                        pnl_cell = calculate_pnl(avg_price, float(ltp), int(filled_qty))
                except KiteException as e:
                    entry_cell = f"ERROR: {str(e)}"
                except Exception as e:
                    entry_cell = f"ERROR: {str(e)}"
        except Exception as e:
            entry_cell = f"ERROR: {e}"
        entry_updates.append([entry_cell])
        pnl_updates.append([pnl_cell])
    while len(price_updates) < 200:
        price_updates.append([""]*9)
    while len(portf_qty_updates) < 200:
        portf_qty_updates.append([""])
    while len(last_updated_updates) < 200:
        last_updated_updates.append([""])
    while len(entry_updates) < 200:
        entry_updates.append([""])
    while len(pnl_updates) < 200:
        pnl_updates.append([""])
    sheet.range("B2:B201").value = portf_qty_updates
    sheet.range("C2:K201").value = price_updates
    sheet.range("P2:P201").value = entry_updates
    sheet.range("Q2:Q201").value = pnl_updates
    sheet.range("R2:R201").value = last_updated_updates

def update_portfolio_sheet(s2, positions, quotes):
    pos_rows = []
    for pos in positions:
        if pos.get('quantity', 0) == 0:
            continue
        exch = pos.get('exchange')
        sym = pos.get('tradingsymbol')
        symbol_key = f"{exch}:{sym}"
        q = quotes.get(symbol_key, {})
        last_price = q.get("last_price", pos.get('last_price', ""))
        pos_rows.append([
            pos.get('instrument_token'),
            sym,
            exch,
            pos.get('quantity'),
            pos.get('average_price'),
            last_price,
            pos.get('unrealised'),
            pos.get('realised'),
            pos.get('day_change')
        ])
    n_positions = len(pos_rows)
    if n_positions > 0:
        s2.range("A2").options(expand='table').value = pos_rows
        if n_positions < MAX_POS_ROWS:
            s2.range(f"A{n_positions+2}:I{MAX_POS_ROWS+1}").value = [[""]*9]*(MAX_POS_ROWS-n_positions)
    else:
        s2.range("A2:I201").value = [[""]*9]*MAX_POS_ROWS
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    s2.range("K1").value = f"Last Updated: {now}"

def update_holdings_sheet(s3, holdings, quotes):
    holding_rows = []
    for h in holdings:
        sym = f"{h.get('exchange')}:{h.get('tradingsymbol')}"
        q = quotes.get(sym, {})
        live_last_price = q.get("last_price", h.get("last_price"))
        holding_rows.append([
            h.get("tradingsymbol"),
            h.get("exchange"),
            h.get("isin"),
            h.get("quantity"),
            h.get("t1_quantity"),
            h.get("average_price"),
            live_last_price,
            h.get("close_price"),
            h.get("pnl"),
            h.get("day_change"),
            h.get("day_change_percentage"),
        ])
    n_holdings = len(holding_rows)
    if n_holdings > 0:
        s3.range("A2").options(expand='table').value = holding_rows
        if n_holdings < MAX_HOLD_ROWS:
            s3.range(f"A{n_holdings+2}:K{MAX_HOLD_ROWS+1}").value = [[""]*11]*(MAX_HOLD_ROWS-n_holdings)
    else:
        s3.range("A2:K201").value = [[""]*11]*MAX_HOLD_ROWS
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    s3.range("L1").value = f"Last Updated: {now}"

def update_settings_sheet(settings_sheet, kite):
    try:
        margins = kite.margins("equity")
        available_margin = margins.get("net", "")
        used_margin = margins.get("utilised", {}).get("debits", "")
        available_cash = margins.get("available", {}).get("live_balance", "")
        settings_sheet.range("A4").value = [
            ["Available Margin", available_margin],
            ["Used Margin", used_margin],
            ["Available Cash", available_cash]
        ]
    except Exception as e:
        print(f"Error fetching/writing margin info: {e}")

def monitor_excel():
    access_token = load_access_token()
    kite = KiteConnect(api_key=API_KEY)
    kite.set_access_token(access_token)
    wb = xw.Book(EXCEL_FILE)
    for name in ['INPUT', 'Portfolio', 'Holdings', 'Orders', 'Settings']:
        if name not in [sh.name for sh in wb.sheets]:
            wb.sheets.add(name)
    input_sheet = wb.sheets['INPUT']
    portfolio_sheet = wb.sheets['Portfolio']
    holdings_sheet = wb.sheets['Holdings']
    orders_sheet = wb.sheets['Orders']
    settings_sheet = wb.sheets['Settings']
    order_id_map = {}
    if settings_sheet.range("A1").value is None:
        settings_sheet.range("A1").value = "Play/Pause"
        settings_sheet.range("B1").value = "No"
        settings_sheet.range("A1:B1").api.Font.Bold = True
        settings_sheet.range("A4").value = [
            ["Available Margin", ""],
            ["Used Margin", ""],
            ["Available Cash", ""]
        ]
    input_headers = [
        "Symbol", "Portfolio Qty", "Open", "High", "Low", "LTP", "Volume", "Avg Price", "Best Bid", "Best Ask", "Prev Close",
        "Qty", "Direction", "Entry Signal", "Exit Signal", "Entry Status", "PnL", "Last Updated",
        "Variety", "Order Type", "Product", "Validity", "Price", "Trigger Price", "TTL Minutes"
    ]
    input_sheet.range("A1").value = input_headers
    input_sheet.range("A1:Y1").api.Font.Bold = True
    portfolio_headers = [
        "Instrument Token", "Trading Symbol", "Exchange", "Quantity",
        "Average Price", "Last Price", "Unrealized PnL", "Realized PnL", "Day's PnL"
    ]
    portfolio_sheet.range("A1").value = portfolio_headers
    portfolio_sheet.range("A1:I1").api.Font.Bold = True
    holdings_headers = [
        "Trading Symbol", "Exchange", "ISIN", "Quantity", "T1 Qty", "Average Price",
        "Last Price (Live)", "Close Price", "PnL", "Day Change", "Day Change %"
    ]
    holdings_sheet.range("A1").value = holdings_headers
    holdings_sheet.range("A1:K1").api.Font.Bold = True
    orders_headers = [
        "Order ID", "Variety", "Status", "Trading Symbol", "Exchange", "Order Type",
        "Product", "Transaction Type", "Quantity", "Price", "Trigger Price",
        "Average Price", "Pending Quantity", "Filled Quantity", "Order Timestamp",
        "Modify", "Cancel", "New Price", "New Trigger" 
    ]
    orders_sheet.range("A1").value = orders_headers
    orders_sheet.range("A1:S1").api.Font.Bold = True
    was_paused = False
    while True:
        try:
            pause_updates = settings_sheet.range("B1").value
            is_paused = pause_updates and str(pause_updates).strip().lower() in ["yes", "y", "true", "1"]
            if is_paused:
                print("Updates paused. Waiting...")
                was_paused = True
                time.sleep(5)
                continue
            force_refresh = was_paused
            if force_refresh:
                print("Resuming from pause, will force refresh Excel data")
                was_paused = False
            update_settings_sheet(settings_sheet, kite)
            input_data = input_sheet.range("A2:A201").value
            input_symbols = [str(s).strip().upper() for s in input_data if s and isinstance(s, str)]
            positions = kite.positions().get('net', [])
            pos_symbols = [f"{pos.get('exchange')}:{pos.get('tradingsymbol')}" for pos in positions if pos.get('exchange') and pos.get('tradingsymbol')]
            holdings = fetch_holdings(kite)
            hold_symbols = [f"{h.get('exchange')}:{h.get('tradingsymbol')}" for h in holdings if h.get('exchange') and h.get('tradingsymbol')]
            all_symbols = list(set(input_symbols + pos_symbols + hold_symbols))
            quotes = {}
            if all_symbols:
                try:
                    quotes = kite.quote(all_symbols)
                except Exception as e:
                    print(f"Error fetching batch quotes: {e}")
            update_input_sheet(input_sheet, kite, order_id_map, quotes, positions)
            update_portfolio_sheet(portfolio_sheet, positions, quotes)
            update_holdings_sheet(holdings_sheet, holdings, quotes)
            update_orders_sheet(orders_sheet, kite)
            process_order_modifications(orders_sheet, kite)
        except Exception as e:
            print(f"Error in monitor loop: {e}")
        time.sleep(5)

if __name__ == "__main__":
    monitor_excel()